In [11]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import dash_leaflet as dl
import pandas as pd
import base64
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################


# Connect to database via CRUD Module
db = AnimalShelter()
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = dash.Dash(__name__)


image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard Jon Wickerd'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'10%', 'width':'10%'}),
    html.Hr(),
    html.Div(
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Option 1: Cats', 'value': 'cat'},
                {'label': 'Option 2: 1 Year Old', 'value': '1_year'},
                {'label': 'Option 3: Pit Bull', 'value': 'pitbull'}
            ],
            value='cat'  # Default value
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Example pagination
        sort_action='native',
        filter_action='native',
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'cat':
        filtered_data = df[df['animal_type'] == 'Cat'].to_dict('records')
    elif filter_type == '1_year':
        filtered_data = df[df['age_upon_outcome_years'] <= 1].to_dict('records')
    elif filter_type == 'pitbull':
        filtered_data = df[df['breed'] == 'Pit Bull'].to_dict('records')
    else:
        filtered_data = df.to_dict('records')  # Default to all data if no match

    return filtered_data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    
    # Count occurrences of each breed
    breed_counts = dff['breed'].value_counts()
    
    # Create pie chart only if there are breeds to show
    if not breed_counts.empty:
        fig = px.pie(names=breed_counts.index, values=breed_counts.values, title='Preferred Animals')
        fig.update_layout(
            height=400,  # Set a fixed height to avoid disproportionate scaling
            width=500,   # Set a fixed width
        )
        return dcc.Graph(figure=fig)
    else:
        return html.Div("No data available for pie chart.")
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(port=8051, mode='external')


[2024-06-18 15:31:25,413] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\jon wicker

[2024-06-18 15:33:03,156] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'age_upon_outcome_years'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\jon wickerd\documents\snhu\cs340\cs-340_local-env\.venv\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_